In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
%matplotlib inline

In [2]:
loads = pd.read_csv('load2011-2023.csv')
weather = pd.read_csv('kalb_all.csv')

In [12]:
weather

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,date
0,11:51 PM,55 °F,40 °F,57 °%,CALM,0 °mph,0 °mph,29.81 °in,0.0 °in,Fair,2001-05-01
1,12:51 AM,54 °F,40 °F,59 °%,CALM,0 °mph,0 °mph,29.81 °in,0.0 °in,Fair,2001-05-01
2,1:51 AM,52 °F,39 °F,61 °%,CALM,0 °mph,0 °mph,29.81 °in,0.0 °in,Fair,2001-05-01
3,2:51 AM,50 °F,39 °F,66 °%,CALM,0 °mph,0 °mph,29.80 °in,0.0 °in,Fair,2001-05-01
4,3:51 AM,48 °F,39 °F,71 °%,CALM,0 °mph,0 °mph,29.80 °in,0.0 °in,Fair,2001-05-01
...,...,...,...,...,...,...,...,...,...,...,...
111354,7:51 PM,52 °F,37 °F,57 °%,S,6 °mph,0 °mph,29.08 °in,0.0 °in,Mostly Cloudy,2023-05-01
111355,8:51 PM,50 °F,37 °F,61 °%,SSE,8 °mph,0 °mph,29.10 °in,0.0 °in,Mostly Cloudy,2023-05-01
111356,9:51 PM,48 °F,38 °F,68 °%,SSE,7 °mph,0 °mph,29.11 °in,0.0 °in,Fair,2023-05-01
111357,10:51 PM,48 °F,39 °F,71 °%,SSE,9 °mph,0 °mph,29.11 °in,0.0 °in,Mostly Cloudy,2023-05-01


In [3]:
weather['date'] = weather.date.apply(lambda x: pd.to_datetime(x).date())
weather['timeest'] = weather.Time.apply(lambda x: pd.to_datetime(x).time())
foo = weather[['date', 'timeest']].astype(str)
weather['timestamp'] = pd.to_datetime(foo['date'] + ' ' + foo['timeest'])
loads['timestamp'] = loads.timestamp.apply(lambda x: pd.to_datetime(x))

In [4]:
weather

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,date,timeest,timestamp
0,11:51 PM,55 °F,40 °F,57 °%,CALM,0 °mph,0 °mph,29.81 °in,0.0 °in,Fair,2001-05-01,23:51:00,2001-05-01 23:51:00
1,12:51 AM,54 °F,40 °F,59 °%,CALM,0 °mph,0 °mph,29.81 °in,0.0 °in,Fair,2001-05-01,00:51:00,2001-05-01 00:51:00
2,1:51 AM,52 °F,39 °F,61 °%,CALM,0 °mph,0 °mph,29.81 °in,0.0 °in,Fair,2001-05-01,01:51:00,2001-05-01 01:51:00
3,2:51 AM,50 °F,39 °F,66 °%,CALM,0 °mph,0 °mph,29.80 °in,0.0 °in,Fair,2001-05-01,02:51:00,2001-05-01 02:51:00
4,3:51 AM,48 °F,39 °F,71 °%,CALM,0 °mph,0 °mph,29.80 °in,0.0 °in,Fair,2001-05-01,03:51:00,2001-05-01 03:51:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111354,7:51 PM,52 °F,37 °F,57 °%,S,6 °mph,0 °mph,29.08 °in,0.0 °in,Mostly Cloudy,2023-05-01,19:51:00,2023-05-01 19:51:00
111355,8:51 PM,50 °F,37 °F,61 °%,SSE,8 °mph,0 °mph,29.10 °in,0.0 °in,Mostly Cloudy,2023-05-01,20:51:00,2023-05-01 20:51:00
111356,9:51 PM,48 °F,38 °F,68 °%,SSE,7 °mph,0 °mph,29.11 °in,0.0 °in,Fair,2023-05-01,21:51:00,2023-05-01 21:51:00
111357,10:51 PM,48 °F,39 °F,71 °%,SSE,9 °mph,0 °mph,29.11 °in,0.0 °in,Mostly Cloudy,2023-05-01,22:51:00,2023-05-01 22:51:00


In [5]:
from sklearn.neighbors import NearestNeighbors

def find_nearest(group, match, groupname):
    nbrs = NearestNeighbors(1).fit(match['timestamp'].values[:, None])
    dist, ind = nbrs.kneighbors(group['timestamp'].values[:, None])

    group['nearesttime'] = match['timestamp'].values[ind.ravel()]
    return group

In [6]:
loads = find_nearest(loads,weather,'timestamp')

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [7]:
loads

,timestamp,name,ptid,load,nearesttime
0,2011-01-01 00:00:00,CAPITL,61757.0,1108.8000,2011-01-01 00:51:00
1,2011-01-01 00:05:00,CAPITL,61757.0,1095.1000,2011-01-01 00:51:00
2,2011-01-01 00:10:00,CAPITL,61757.0,1104.7000,2011-01-01 00:51:00
3,2011-01-01 00:13:37,CAPITL,61757.0,1087.6000,2011-01-01 00:51:00
4,2011-01-01 00:14:49,CAPITL,61757.0,1085.4000,2011-01-01 00:51:00
...,...,...,...,...,...
1377972,2023-12-31 23:35:00,CAPITL,61757.0,1197.5303,2023-05-01 23:51:00
1377973,2023-12-31 23:40:00,CAPITL,61757.0,1209.7126,2023-05-01 23:51:00
1377974,2023-12-31 23:45:00,CAPITL,61757.0,1192.3306,2023-05-01 23:51:00
1377975,2023-12-31 23:50:00,CAPITL,61757.0,1196.0747,2023-05-01 23:51:00


In [8]:
full = loads.merge(weather, left_on='nearesttime', right_on='timestamp')
print(full.columns)
print(full)

full = full[['timestamp_x', 'load', 'nearesttime', 'Temperature', \
            'Dew Point', 'Humidity', 'Pressure', 'Wind', 'Wind Speed', \
            'Precip.']].rename(columns={'timestamp_x': 'timestamp', 'nearesttime':'weathertime', 'Precip.': 'precipitation'})

Index(['timestamp_x', 'name', 'ptid', 'load', 'nearesttime', 'Time',
       'Temperature', 'Dew Point', 'Humidity', 'Wind', 'Wind Speed',
       'Wind Gust', 'Pressure', 'Precip.', 'Condition', 'date', 'timeest',
       'timestamp_y'],
      dtype='object')
                timestamp_x    name     ptid       load         nearesttime  \
0       2011-01-01 00:00:00  CAPITL  61757.0  1108.8000 2011-01-01 00:51:00   
1       2011-01-01 00:05:00  CAPITL  61757.0  1095.1000 2011-01-01 00:51:00   
2       2011-01-01 00:10:00  CAPITL  61757.0  1104.7000 2011-01-01 00:51:00   
3       2011-01-01 00:13:37  CAPITL  61757.0  1087.6000 2011-01-01 00:51:00   
4       2011-01-01 00:14:49  CAPITL  61757.0  1085.4000 2011-01-01 00:51:00   
...                     ...     ...      ...        ...                 ...   
1378143 2023-12-31 23:35:00  CAPITL  61757.0  1197.5303 2023-05-01 23:51:00   
1378144 2023-12-31 23:40:00  CAPITL  61757.0  1209.7126 2023-05-01 23:51:00   
1378145 2023-12-31 23:45:00  CA

In [9]:
# full.to_csv('full2012.csv',encoding='utf-8-sig', index=False)

In [10]:
# full = pd.read_csv('full2012.csv')

In [11]:
pday = pd.Timedelta('1 day')

def get_prev_days(x, n_days):
    try:
        lo = full[full.timestamp == x - n_days*pday].load.values[0]
    except:
        lo = full[full.timestamp == x].load.values[0]
    return lo 

In [12]:
full['dow'] = full.timestamp.apply(lambda x: x.dayofweek)
full['doy'] = full.timestamp.apply(lambda x: x.dayofyear)
full['day'] = full.timestamp.apply(lambda x: x.day)
full['month'] = full.timestamp.apply(lambda x: x.month)
full['hour'] = full.timestamp.apply(lambda x: x.hour)
full['minute'] = full.timestamp.apply(lambda x: x.hour*60 + x.minute)

full['t_m24'] = full.timestamp.apply(get_prev_days, args=(1,))
full['t_m48'] = full.timestamp.apply(get_prev_days, args=(2,))
full['tdif'] = full['load'] - full['t_m24']

In [13]:
full['Temperature'] = full['Temperature'].map(lambda x: x[:-2])
full['Dew Point'] = full['Dew Point'].map(lambda x: x[:-2])
full['Humidity'] = full['Humidity'].map(lambda x: x[:-2])
full['Pressure'] = full['Pressure'].map(lambda x: x[:-3])
full['Wind Speed'] = full['Wind Speed'].map(lambda x: x[:-4])
full['precipitation'] = full['precipitation'].map(lambda x: x[:-3])

In [14]:
full

,timestamp,load,weathertime,Temperature,Dew Point,Humidity,Pressure,Wind,Wind Speed,precipitation,dow,doy,day,month,hour,minute,t_m24,t_m48,tdif
0,2011-01-01 00:00:00,1108.8000,2011-01-01 00:51:00,40,29,65,29.77,S,8,0.0,5,1,1,1,0,0,1108.8000,1108.8000,0.0000
1,2011-01-01 00:05:00,1095.1000,2011-01-01 00:51:00,40,29,65,29.77,S,8,0.0,5,1,1,1,0,5,1095.1000,1095.1000,0.0000
2,2011-01-01 00:10:00,1104.7000,2011-01-01 00:51:00,40,29,65,29.77,S,8,0.0,5,1,1,1,0,10,1104.7000,1104.7000,0.0000
3,2011-01-01 00:13:37,1087.6000,2011-01-01 00:51:00,40,29,65,29.77,S,8,0.0,5,1,1,1,0,13,1087.6000,1087.6000,0.0000
4,2011-01-01 00:14:49,1085.4000,2011-01-01 00:51:00,40,29,65,29.77,S,8,0.0,5,1,1,1,0,14,1085.4000,1085.4000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378143,2023-12-31 23:35:00,1197.5303,2023-05-01 23:51:00,47,38,71,29.10,S,6,0.0,6,365,31,12,23,1415,1180.7429,1129.8756,16.7874
1378144,2023-12-31 23:40:00,1209.7126,2023-05-01 23:51:00,47,38,71,29.10,S,6,0.0,6,365,31,12,23,1420,1186.3677,1124.2838,23.3449
1378145,2023-12-31 23:45:00,1192.3306,2023-05-01 23:51:00,47,38,71,29.10,S,6,0.0,6,365,31,12,23,1425,1188.1533,1120.9103,4.1773
1378146,2023-12-31 23:50:00,1196.0747,2023-05-01 23:51:00,47,38,71,29.10,S,6,0.0,6,365,31,12,23,1430,1163.1700,1122.9865,32.9047


In [16]:
full.to_csv('full2011-2020_features.csv',encoding='utf-8-sig', index=False)

In [3]:
full = pd.read_csv('full2011-2020_features.csv')

In [5]:
full['timestamp'] = full.timestamp.apply(lambda x: pd.to_datetime(x))
full['year'] = full.timestamp.apply(lambda x: x.year)

In [6]:
full.to_csv('full2011-2020_features.csv',encoding='utf-8-sig', index=False)